In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_huggingface import HuggingFaceEmbeddings
from chroma_store import chroma_store

from langchain_community.document_loaders import PyPDFLoader

file_path = "./constitution-1958.pdf" # https://www.legifrance.gouv.fr/download/pdf/legiOrKali?id=LEGITEXT000006072050.pdf&size=7%20Mo&pathToFile=/LEGI/TEXT/00/00/06/07/20/50/LEGITEXT000006072050/LEGITEXT000006072050.pdf&title=Code%20du%20travail
loader = PyPDFLoader(file_path)
data = loader.load()
print("Document loaded")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)
chroma_store.add_documents(documents=all_splits)
print("Vectors stored")


/home/unix/.pyenv/versions/t-esp/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/unix/.pyenv/versions/t-esp/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Document loaded
Vectors stored


In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


from langchain.callbacks.manager import CallbackManager
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
Vous êtes un assistant qui rend le droit français plus accessible au grand public.
Utilisez les informations suivantes pour répondre à la question posée.
L'exactitude est primordiale : si vous ne connaissez pas la réponse, indiquez simplement que vous ne savez pas.
La réponse doit être en français et comporter au maximum 800 caractères.

<context>
{context}
</context>

Répondez à la question suivante:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)
llm = Ollama(model="mistral:latest", verbose=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | llm
    | StrOutputParser()
)
print("Chain loaded")

question = "Quelle est la langue de la république ?"

docs = chroma_store.similarity_search(question)
print(docs)
print("Context loaded")

# Run
print("Running chain")
chain.invoke({"context": docs, "question": question})

Chain loaded
[Document(metadata={'page': 2, 'source': './constitution-1958.pdf'}, page_content="République et l'élection de son successeur.\nAr ticle 8\nLe Président de la République nomme le Premier ministre. Il met fin à ses fonctions sur la présentation par celui-ci de la démission du\nGouvernement."), Document(metadata={'page': 2, 'source': './constitution-1958.pdf'}, page_content="République ou durant la période qui s'écoule entre la déclaration du caractère définitif de l'empêchement du Président de la\nRépublique et l'élection de son successeur.\nAr ticle 8"), Document(metadata={'page': 0, 'source': './constitution-1958.pdf'}, page_content='professionnelles et sociales.\nTitr e pr emier : De la sou v er ain eté (Ar ticles 2 à 4)\nAr ticle 2\nLa langue de la République est le français.\nL\'emblème national est le drapeau tricolore, bleu, blanc, rouge.\nL\'hymne national est la " Marseillaise ".'), Document(metadata={'page': 2, 'source': './constitution-1958.pdf'}, page_content="R